<a href="https://colab.research.google.com/github/sathyanarayanajammala/GenAI/blob/main/Langchain_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative AI and Prompt Engineering
## A Program by IISc and TalentSprint



## **LangChain: Chains**

In [ ]:
!pip install openai
!pip install langchain
!pip install langchain-openai

In [ ]:
import openai
import os

In [ ]:
f = open('/content/openapi_key.txt')
api_key = f.read().strip()          # Remove Blank Spaces
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

### **Most Basic chain**
[LCEL:Getting Started](https://python.langchain.com/v0.1/docs/expression_language/get_started/)

**Example:1**

Notice: `.from_template`

In [ ]:
llm_model = "gpt-3.5-turbo" # This is a chat model
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [ ]:
prompt1 = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [ ]:
prompt1

In [ ]:
chain1 = prompt1 | llm |StrOutputParser() # Try with or without-> |StrOutputParser()

**LCEL(LangChain Expression Language)**--> uses pipes(|):  When we call invoke() on the chain, the input is first passed to prompt, output of that is passed to the llm. It's just the combination of the LLM
and the prompt. But now this chain will let us run through the
prompt and into the LLM in a sequential manner.

In [ ]:
product = "mobile"
result= chain1.invoke(product)
result

**Example:2**

Notice: `.from_messages`

In [ ]:
prompt2=ChatPromptTemplate.from_messages([("system","You are a knowladgeable historian" ),
                                          ("human","say 3 lines about {input}")])
prompt2

In [ ]:
chain2 = prompt2| llm |StrOutputParser() # LCEL --> LangChain Expression Language

In [ ]:
event = "India Gate"
result= chain2.invoke({'input':event})
result

### **Runnable**

A Runnable is a unit of execution in the LangChain framework. It represents a specific task or operation that can be performed.
Examples of Runnables include data transformations, computations, or any other operation that can be expressed in the LangChain expression language.

**Example 1: [Runnable Lambdas](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html)**

The RunnableLambda is a LangChain abstraction that allows us to turn Python functions into pipe-compatible functions, similar to the Runnable class.

In [ ]:
from langchain_core.runnables import RunnableLambda

In [ ]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with RunnableLambda
chain = RunnableLambda(add_five) | RunnableLambda(multiply_by_two)


In [ ]:
chain.invoke(3)

In [ ]:
# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
sequence.invoke(1)

**Example 2 : [RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html)** on its own allows you to pass inputs unchanged. This typically is **used in conjuction with [RunnableParallel](https://python.langchain.com/v0.1/docs/expression_language/interface/#parallelism)** to pass data through to a new key in the map.

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified = lambda x:   x["num"] + 1  ,
)

In [ ]:
runnable.invoke({"num": 3})

**Example 3: RunnableMap**: Parallel Execution -
The RunnableMap class runs a mapping of Runnables in parallel. It takes a list of inputs and applies each Runnable to its corresponding input.
The result is a mapping of the outputs from each Runnable.
Essentially, it allows you to process multiple inputs concurrently and collect their outputs in a structured way.

In [ ]:
## Fictional character
context1 =  """ Dr. Ananya Verma, a legendary Indian physicist born
in Kolkata in 1960, has made significant contributions to theoretical physics.
 Her achievements include the discovery of the Bose–Verma condensate, formulation
  of quantum entanglement theorems, and resolution of the Verma–Hawking black hole paradox.
  Dr. Verma’s brilliance has earned her praise from fellow scientists,
 and her legacy continues to inspire generations of physicists worldwide. """

question1 ="What are few discoveris by Dr. Ananya Verma?"

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
prompt

In [ ]:
from langchain_core.runnables import RunnableMap

In [ ]:
chain = RunnableMap(  {"context": lambda x: x["context"], "question": lambda x: x["question"]}   )   | prompt | llm | StrOutputParser()
chain

In [ ]:
chain = {"context": lambda x: x["context"], "question": lambda x: x["question"]} | prompt | llm | StrOutputParser()
chain

In [ ]:
chain.invoke(  {"context":context1,"question": question1}  )

### **Complex - Sequential Chain**
**Example :1**

In [ ]:
review = """Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?"""

In [ ]:
llm_model = "gpt-3.5-turbo" # This is a chat model
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [ ]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = first_prompt|llm|StrOutputParser()   # output_key="English_Review"

In [ ]:
eng_review=chain_one.invoke(review)
eng_review

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = second_prompt|llm|StrOutputParser()  # output_key="summary"

In [ ]:
chain_two.invoke(eng_review)

In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = third_prompt|llm|StrOutputParser() # output_key="language"

In [ ]:
chain_three.invoke(review)

In [ ]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {Summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = fourth_prompt|llm|StrOutputParser() # output_key="followup_message"


In [ ]:
chain_four.invoke({"Summary":'The reviewer is disappointed with the taste and quality of the product, questioning if it is an old batch or counterfeit.',
                   "language":'French'})

In [ ]:
from langchain_core.runnables import RunnableParallel

runnable1 = RunnableParallel({
    "Summary": chain_one | chain_two,
    "language": chain_three,
   })

In [ ]:
runnable1.invoke(review)

In [ ]:
final_chain= RunnableParallel({"Summary": chain_one | chain_two,"language": chain_three,}) |  {'followup_message':chain_four}


In [ ]:
#from langchain.callbacks.tracers import ConsoleCallbackHandler
final_chain.invoke(review) #,config={'callbacks': [ConsoleCallbackHandler()]})